In [1]:
from  llmcoder import LLMCoder

from tqdm.auto import tqdm

from concurrent.futures import ThreadPoolExecutor

/home/psaegert/miniconda3/envs/llmcoder/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
failing_example = """import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from dynaconf import Dynaconf

from llmcoder.utils import get_config_dir, get_data_dir

# Get all configs
config_dir = get_config_dir()
config_file_list = sorted(file for file in os.listdir(config_dir) if file.endswith(".yaml"))
config_list = """

gt_output = """[Dynaconf(settings_file=os.path.join(config_dir, config_name)) for config_name in config_file_list]"""

In [3]:
N_REPEAT = 1000

In [4]:
for r in tqdm(range(N_REPEAT)):
    llmcoder = LLMCoder(analyzers=["mypy_analyzer_v1", "jedi_analyzer_v1"], n_procs=16, verbose=False)

    with ThreadPoolExecutor(max_workers=llmcoder.n_procs) as executor:
        # Create a mapping of future to completion choice
        choice_to_future = {
            i: executor.submit(llmcoder.run_analyzers, failing_example, gt_output)
            for i in range(llmcoder.n_procs)
        }

        # Retrieve results in the order of valid_choices
        analysis_results_list = []
        for i in range(len(choice_to_future)):
            analysis_results_list.append(choice_to_future[i].result())

 58%|█████▊    | 580/1000 [15:24<11:09,  1.59s/it]


KeyboardInterrupt: 

In [6]:
failing_example = """import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from dynaconf import Dynaconf

from llmcoder.utils import get_config_dir, get_data_dir

# Get all configs
config_dir = get_config_dir()
config_file_list = sorted(file for file in os.listdir(config_dir) if file.endswith(".yaml"))
config_list = """

gt_outputs = [
    """[Dynaconf(settings_file=os.path.join(config_dir, config_name)) for config_name in config_file_list]""",
    """[Dynaconf(os.path.join(config_dir, config_name)) for config_name in config_file_list]""",
    """[Dynaconf(settings_file=config_dir for config_name in config_file_list]""",
    """Dynaconf(settings_file=os.path.join(config_dir, config_name))""",
    """Dynaconf(
        settings_file=os.path.join(config_dir, config_name)
        )""",
    """Dynaconf(
        settings_file=os.path.join(config_dir, config_name)
        )""",
]

In [7]:
for r in tqdm(range(N_REPEAT)):
    llmcoder = LLMCoder(analyzers=["mypy_analyzer_v1", "jedi_analyzer_v1"], n_procs=16, verbose=False)

    with ThreadPoolExecutor(max_workers=llmcoder.n_procs) as executor:
        # Create a mapping of future to completion choice
        choice_to_future = {
            i: executor.submit(llmcoder.run_analyzers, failing_example, output)
            for i, output in enumerate(gt_outputs)
        }

        # Retrieve results in the order of valid_choices
        analysis_results_list = []
        for i in range(len(choice_to_future)):
            analysis_results_list.append(choice_to_future[i].result())

 74%|███████▍  | 739/1000 [13:45<04:51,  1.12s/it]


KeyboardInterrupt: 